In [166]:
import openai,time,pandas,configur
from openai import OpenAI
from statistics import mean,median
import pandas as pd

In [345]:
def ask_chatgpt(prompt):
    try:
        system_role = {"role": "system", "content": "After every single ; write [Buy/Sell/Neither] [Ticker/ TICKER NOT FOUND] "}
        # Sends the prompt to ChatGPT using the chat completion API
        client = OpenAI(api_key = configur.thepass['gpt_api'])    
        completion = client.chat.completions.create(
                      model="gpt-3.5-turbo",
                      messages=[system_role, {"role": "user", "content": prompt}])
        # Extracts the text from the response object
        answer_text = completion.choices[0].message.content
        print("ChatGPT:", answer_text)
        return answer_text
    except openai.APIError as e:
        print(f"API Error: {e}")
        time.sleep(60)
        return ask_chatgpt(prompt)
    except openai.APIConnectionError as e:
        print(f"Connection Error: {e}")
        return None
    except openai.RateLimitError as e:
        print(f"Rate Limit Error: {e}")
        time.sleep(60)
        return ask_chatgpt(prompt)

In [346]:
def read_msg_df(csv_file):
    msg_df = pandas.read_csv(f'{csv_file}',encoding='utf-8',index_col = 0)
    msg_df.head(10)
    return msg_df


In [353]:
def send_split_data():
    csv_file = r'C:\Users\amoog\Desktop\Project_X\Project_X\disc_msg_df.csv'
    df = read_msg_df(csv_file)
    df = df.head(30)  # Limiting the data for processing.

    responses = []
    response_indices = []

    accumulated_messages = []
    accumulated_indices = []
    sum_so_far = 0

    for index, msg in df['Message'].iteritems():
        word_count = len(msg.split())
        sum_so_far += word_count
        accumulated_messages.append(msg)
        accumulated_indices.append(index)

        if sum_so_far > 1000:
            response_text = ask_chatgpt('; '.join(accumulated_messages))
            split_responses = response_text.split('\n') if response_text else ["Response unavailable."]*len(accumulated_messages)
            print("Responses:", split_responses)  # Debugging output
            responses.extend(split_responses)
            response_indices.extend(accumulated_indices)
            accumulated_messages = []
            accumulated_indices = []
            sum_so_far = 0

    # Handle the last batch if it exists
    if accumulated_messages:
        response_text = ask_chatgpt('; '.join(accumulated_messages))
        split_responses = response_text.split('\n') if response_text else ["Response unavailable."]*len(accumulated_messages)
        responses.extend(split_responses)
        response_indices.extend(accumulated_indices)

    # Create a DataFrame from responses and their corresponding indices
    response_df = pd.DataFrame({'ChatGPT_Response': responses}, index=response_indices)

    # Join the response DataFrame to the original DataFrame
    df = df.join(response_df, how='left')
    return df

# Execute the function and output the DataFrame
df = send_split_data()


ChatGPT: [Buy] SPY  
[Neither] ALGN  
[Buy] COST  
[Neither] ADBE  
[Neither] MLKN  
[Neither] NIO  
[Neither] PLTR  
[TICKER NOT FOUND] JD  
[Sell] SNOW  
[Neither] TSLA  
[Buy] SAVA  
[Neither] LCID  
[Buy] PLTR  
[TICKER NOT FOUND] PENNIES  
[TICKER NOT FOUND] DWAC  
[Neither] BGCL  
[Neither] CHWY  
[Neither] PSQH  
[Neither] FACT  
[Sell] AAPL  
[Neither] RAD  


ValueError: Shape of passed values is (21, 1), indices imply (30, 1)

In [352]:
df.head(60)

,Username,Message,Timestamp,ChatGPT_Response
0,waysotheraccount,spy 451c 0dte at 16 worth a shot,2023-08-07T17:24:52.421000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
1,waysotheraccount,algn seems a candidate for a bounce off a huge...,2023-10-26T13:49:15.147000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
2,Bullish B Signals,ENTERING COST 695 1/19 CALLS @ 048,2024-01-19T20:22:59.952000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
3,waxui,BULLPRINTERPlanned Plays are posted above^ Per...,2023-04-05T23:31:37.725000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
4,waysotheraccount,watch mlkn on an er move up or down but could ...,2023-07-12T18:56:26.367000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
5,waysotheraccount,I'll hope you guys are making profit Remember ...,2023-07-13T16:11:49.564000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
6,Market Bishop,Setting trims,2024-03-28T14:07:38.157000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
7,Prodigy,SPY 514C 0DTE - I’m going in light here super ...,2024-04-11T14:20:54.926000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
8,waysotheraccount,still holding those long rum calls and nio,2024-03-12T19:31:30.606000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
9,waysotheraccount,pltr extreme lotto play 20c at 07 selling if i...,2023-07-17T17:13:22.655000+00:00,[Neither] SPY \n[Buy] ALGN \n[Buy] COST \n[...
